# Scrape Amazon Data using Python

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [14]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string
# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={"class":'a-offscreen'}).string.strip()

    except:
            price = ""

    return price
# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={"class":'a-icon a-icon-star a-star-4 cm-cr-review-stars-spacing-big'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [15]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=gaming&ref=nb_sb_noss"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    #importing to csv file
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [16]:
amazon_df

,title,price,rating,reviews,availability
0,"STGAubron Gaming Desktop PC,Intel Core i7 up t...",$799.99,4.0 out of 5 stars,25 ratings,In Stock
1,"ASUS 2022 TUF Dash F15 Gaming Laptop, 15.6"" FH...","$1,399.99",3.9 out of 5 stars,5 ratings,Only 4 left in stock - order soon
2,Large RGB Gaming Mouse Pad -15 Light Modes Tou...,$8.99,4.7 out of 5 stars,"2,164 ratings",In Stock
3,Skytech Azure Gaming PC Desktop – Intel Core i...,"$1,799.99",4.4 out of 5 stars,434 ratings,Only 19 left in stock (more on the way)
4,Logitech for Creators Blue Snowball iCE USB Mi...,$46.49,4.6 out of 5 stars,"35,390 ratings",In Stock
...,...,...,...,...,...
69,"Redragon M612 Predator RGB Gaming Mouse, 8000 ...",$19.99,4.6 out of 5 stars,"1,755 ratings",In Stock
70,"MSI Thin GF63 15.6"" 144Hz Gaming Laptop: 12th ...",$999.00,4.5 out of 5 stars,"1,816 ratings",In Stock
71,"soundcore VR P10 Wireless Gaming Earbuds, Meta...",$79.00,4.3 out of 5 stars,781 ratings,In Stock
72,"Victus by HP 15 Gaming Laptop, NVIDIA GeForce ...",$738.99,4.3 out of 5 stars,231 ratings,Only 2 left in stock - order soon
